In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np

from matplotlib import pyplot as plt
import scipy
from scipy.ndimage import gaussian_filter


import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

import cv2
import os

import scipy.misc
#import skimage.filter

import parmap

import numpy as np
import matplotlib.pyplot as plt
import scipy.misc


Autosaving every 180 seconds


In [2]:

def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

In [3]:
#fname_predictions = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/predictions.npy'
fname_predictions = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/predictions_10k_new.npy'

predictions = np.load(fname_predictions)[:,:,:,:10]
print (predictions.shape)

# ids = [2, 50, 111]
# plt.imshow(predictions[0].sum(2))
# plt.show()

(10000, 128, 160, 10)


In [4]:
# # load surfaces
# id_ = 100

# fname_surfaces = fname_predictions[:-4]+'_surfaces.npy'
# surfaces = np.load(fname_surfaces)
# print (surfaces.shape)
# plt.imshow(surfaces[id_])

# # load assembled animals
# traces_cc = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz')

# tracesx = traces_cc['tracesx']
# print (tracesx.shape)
# tracesy = traces_cc['tracesy']
# plt.scatter(tracesx[:,id_]/8, tracesy[:,id_]/8, c='red')

# plt.show()



In [5]:
####################################################################
#################### CONVERT PREDICTIONS TO SURFCACES ##############
####################################################################

fname_surfaces = fname_predictions[:-4]+'_surfaces.npy'

if os.path.exists(fname_surfaces)==False:

    n_frames = predictions.shape[0]
    surfaces = np.zeros((n_frames,predictions.shape[1],predictions.shape[2]),'float32')
    #predictions = np.zeros((n_frames,128,160,14),'float32')

    threshold = 0.5
    features = np.arange(10)
    for n in range(n_frames):
        if n%100==0:
            print (n)
        sums = np.zeros((predictions.shape[1],predictions.shape[2]),'float32')
        #for k in range(14):
        for k in features:
            temp= predictions[n,:,:,k]
            temp = (temp -np.min(temp))/ (np.max(temp)-np.min(temp))
            temp = 1 / (np.exp(-temp) + 1)  # confidence map transformation as per Anqi's sugestion

            idx = np.where(temp<threshold)
            temp[idx]=0
            sums+=temp
        surfaces[n]= sums

    np.save(fname_surfaces, surfaces)

else:
    surfaces = np.load(fname_surfaces)

print (" Final surfaces: ", surfaces.shape)
print ("DONE")

 Final surfaces:  (10000, 128, 160)
DONE


In [6]:
####################################################################
#################### EXTRACT PATCHES VIA WATERSHED ##############
####################################################################

def extract_patches_watershed_parallel(image, min_pixels):
#img_array1 = []
#for ctr in range(surfaces.shape[0]):

    #if ctr%100==0:
    #    print ("frame; ", ctr)
    #temp = predictions[ctr].sum(2)
    #temp = (temp-np.min(temp))/(np.max(temp)-np.min(temp))
    #image = temp.copy()*10
    #image=surfaces[ctr]

    #binarize = image>0.5
    #bin_neg = image<1
    #image = binarize
    
    #image = binarize.copy()
    if False:
        thresh = np.max(image)//2
        idx = np.where(image<thresh)
        image[idx]=0
        
    if True:
        distance = ndi.distance_transform_edt(image)
        local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3, 3)),
                                    labels=image)
        markers = ndi.label(local_maxi)[0]
        labels = watershed(-distance, markers, mask=image)

    else:
        # Now we want to separate the two objects in image
        # Generate the markers as local maxima of the distance to the background
        distance = ndi.distance_transform_edt(image)
        local_maxi = peak_local_max(distance, indices=False, 
                                    footprint=np.ones((3, 3)),
                                    labels=image)
        markers = ndi.label(local_maxi)[0]
        labels = watershed(-distance, markers, mask=image)
        #labels = watershed(-surfaces[ctr], markers, mask=image)

    
    # REMOVE PATCHES BELOW MIN # PIXEL THRESHOLD
    #print (np.unique(labels))
    for k in np.unique(labels):
        idx = np.where(labels==k)
        if idx[0].shape[0]<min_pixels:
            labels[idx]=0        
    
    return labels


fname_watershed = fname_predictions[:-4]+"_watershed.npy"

if os.path.exists(fname_watershed)==False:
    
    surfaces = np.load(fname_surfaces)
    min_pixels = 50
    images = surfaces[:250]
    #plt.imshow(images[0])
    result = parmap.map(extract_patches_watershed_parallel, 
                        images, 
                        min_pixels,
                        pm_pbar=True,
                        pm_processes=8)

    result = np.array(result)
    print ("Watershed patches: ", result.shape)
    np.save(fname_watershed, result)

else:
    result = np.load(fname_watershed)
    
print ("DONE; water shed: ", result.shape)
    
    

DONE; water shed:  (10000, 128, 160)


In [7]:
# 

#test watershed again


import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

print (predictions.shape)
#image = predictions[0].sum(2)

root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/'

fname_video = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'
original_vid = cv2.VideoCapture(fname_video)
    
#predictions = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/predictions_test.npy')
#predictions = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/predictions.npy')
start = 0
number = 10000
n_animals = 6
features = np.arange(10)
threshold = 0.5

plotting = False

original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

from tqdm import tqdm


ids=np.arange(start,start+number,1)

fname_watershed = fname_predictions[:-4]+"_watershed.npy"
print (fname_watershed)

if os.path.exists(fname_watershed)==False:
    label_array = []
    for ctr, id_ in tqdm(enumerate(ids)):

        # Anqi's sigmoid transformation
        pred_s = predictions[id_]
        pred_s = 1 / (np.exp(-pred_s) + 1)  # confidence map

        surface = np.zeros((predictions.shape[1],predictions.shape[2]),'float32')
        #for k in range(14):
        for k in features:
            temp= pred_s[:,:,k]

            # normalize data before adding together
            temp = (temp -np.min(temp))/ (np.max(temp)-np.min(temp))

            # additional step suggested by Anqi; possible to add it to final image also
            #temp = 1 / (np.exp(-temp) + 1)  # confidence map transformation as per Anqi's sugestion

            idx = np.where(temp<threshold)
            temp[idx]=0
            surface+=temp

        image = surface.copy()

        #print ("image; ", image.shape)


        thresh_flex = 0.2 * np.max(image)
        image = gaussian_filter(image, sigma=1.5)
        ctr2 = 0
        while True:
            binarize = image.copy()>thresh_flex

            #binarize = image>(np.max(image)/1.2)
            #bin_neg = image>np

            distance = ndi.distance_transform_edt(binarize)
           # print ("distance: ", distance.shape)
            local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((1, 1)),
                                        labels=binarize)
            markers = ndi.label(local_maxi)[0]
            labels = watershed(-distance, 
                               markers, 
                               mask=binarize)

            #print (" ctr: ", ctr,  "  # animalsL ", np.unique(labels).shape[0], ctr2)
            if np.unique(labels).shape[0]<n_animals and ctr2<15:
                thresh_flex = thresh_flex*.9
            else:
                break

            ctr2+=1

        # REMOVE PATCHES BELOW MIN # PIXEL THRESHOLD
        #print (np.unique(labels))
        min_pixels = 5

        while True:
            for k in np.unique(labels):
                idx = np.where(labels==k)
                if idx[0].shape[0]<min_pixels:
                    labels[idx]=0

            if np.unique(labels).shape[0]<(n_animals+2):
                break
            min_pixels = min_pixels*1.5

        #ax=plt.subplot(2,2,1)
        #plt.imshow(distance)


        if plotting:
            if  number==10:
                ax=plt.subplot(2,5,ctr+1)
            else:
                ax=plt.subplot(10,10,ctr+1)

            labels = labels*(255/np.max(labels))
            ret, frame = original_vid.read()
            img_out = np.vstack((labels, frame[::8,::8].mean(2)))
            plt.imshow(img_out)
            plt.ylabel(str(id_))
            plt.xticks([])
            plt.yticks([])

        label_array.append(labels)
    if plotting:    
        plt.show()
    label_array = np.array(label_array)
    np.save(fname_watershed, label_array)
else:
    label_array=np.load(fname_watershed)
    
print ("watershed : ", label_array.shape)

(10000, 128, 160, 10)
/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/predictions_10k_new_watershed.npy
watershed :  (10000, 128, 160)


In [8]:
############################################################################
########## TIME_BASED CONNECTED COMPONNET 1: FIND OVERLAPS OVER TIME #######
############################################################################

def intersect2D(a, b):
  """
  Find row intersection between 2D numpy arrays, a and b.
  Returns another numpy array with shared rows
  """
  return np.array([x for x in set(tuple(x) for x in a) & set(tuple(x) for x in b)])

# 
surfaces_watershed = np.load(fname_predictions[:-4]+"_watershed.npy")

# identify periods of time where the identity does not switch
locs = [[0, 215], 
        [2180, 2468], 
        [4400, 4500],
        [5376, 5583],
        [0, 3000],
        [0,10000]
       ]

# SELECT A SEGMENT OF TIME
frames = np.arange(locs[5][0], 
                   locs[5][1],1)
print ("Frames; ", frames)

# 
fname_matches = fname_predictions[:-4]+"_matches_"+str(frames[0])+"_"+str(frames[-1])+".npy"

if os.path.exists(fname_matches)==False:
#if True:
    min_overlap_percentage = 0.5
    min_overlap_pixels = 50
    matches = []
    
    # add ids of the first frame as-is
    matches.append(np.unique(surfaces_watershed[frames[0]]))  #Pad 

    max_patch = 500 # max size of a patch to exclude background matching
    
    # loop over all frames
    for ctr_k, k in enumerate(frames[1:]):  #skip first match
        matches.append([])
        if k%10==0:
            print ("Frame: ", k)
            
        # find patch 
        im = surfaces_watershed[k]

        # 
        ids = np.unique(im)
        for ctr, id_ in enumerate(ids):
            matches[ctr_k+1].append([])
            #if id_==0:
            #    continue
            idx1 = np.array(np.where(im==id_)).T

            match_id = None

            # exclude large patches from analysis:
            if idx1.shape[0]<max_patch:
                # loop over previous frame blobs
                max_match = 0
                for p in np.unique(surfaces_watershed[k-1]):
                    idx2 = np.array(np.where(surfaces_watershed[k-1]==p)).T
                    if idx2.shape[0]>max_patch:
                        continue
                    
                    # don't allow for matches to the background for IDs that were pev missed;
                    i = intersect2D(idx1,idx2)

                    #if i.shape[0]>max_match and i.shape[0]>(min_overlap*i.shape[0]):
                    if i.shape[0]>max_match and i.shape[0]>min_overlap_pixels:
                        match_id = p
                        max_match = i.shape[0]

            matches[ctr_k+1][ctr].append(match_id)
        
            #print("k :", k, ' ctr: ', ctr, " match_id: ", match_id)
    np.save(fname_matches, matches)
else:
    matches = np.load(fname_matches, allow_pickle=True)
    
print ("matches; ", len(matches))
print ("DONE")

Frames;  [   0    1    2 ... 9997 9998 9999]
matches;  10000
DONE


In [17]:
##################################################################
########## TIME_BASED CONNECTED COMPONNET 2: EXTRACT NETWORKS ####
##################################################################
import functools
import operator

clrs = ['red','blue','cyan','green','yellow','pink','magenta','black','lightgreen','lightblue']

def find_next_id(tracker):
    
    temp = functools.reduce(operator.iconcat, tracker, [])
    temp = [x for x in temp if x]

    temp2 = np.sort(np.unique(temp))
    for k in range(1, np.max(temp2)+1):
        idx = np.where(temp2==k)[0]
        #print ("K: ", k, temp2)
        if idx.shape[0]==0:
            return k
            break

    #print ("Missing val: ", k, "  from : ", temp2)
    #print ("k :", k)
    #print ('')

    return k+1


fname_tracker = fname_matches[:-4]+"_tracker.npy"
if True:
    frames = np.arange(400)
#if os.path.exists(fname_tracker)==False:
    chains = []
    # 
    chain_ids = np.unique(matches[0])
    print ("Chain IDS: ", chain_ids)

    max_network_id = np.max(chain_ids)+1

    plotting = True

    print ("PLOTING: ", plotting)
    if plotting:
        fig=plt.figure(figsize=(40,20))
        ax1=plt.subplot(1,1,1)
        #ax2=plt.subplot(2,1,2)

    tracker = []
    tracker.append(chain_ids)

    for ctr_k, k in enumerate(range(frames[0]+1,frames[-1], 1)):

        prev_ids = np.unique(surfaces_watershed[k-1])
        local_ids = np.unique(surfaces_watershed[k])

        # add a list to contain all the chain ids for this frame:
        tracker.append([])
        for l in range(local_ids.shape[0]):
            tracker[ctr_k+1].append([])

        #max_network_id = np.max(tracker[:,k-1])+1

        #for p in range(len(matches[k])):
        for ctr, p in enumerate(range(len(local_ids))):
            idx0 = np.where(surfaces_watershed[k]==local_ids[p])

            # SKIP THE DEFAULT BACKGROUND PATCH
            if idx0[0].shape[0]>500:
                continue

            prev_id = np.array(matches[ctr_k+1][p]).squeeze()
            current_id = local_ids[p]
            #current_id = ctr
    #         print ([k-1, k])
    #         print ("prev id: ", prev_id)
    #         print (current_id)

            # if patch matches a prev_id
            #print ("k: ", k, " prev_ID: ", prev_id)
            if prev_id != None:
                # inherit the chain ID of the previous val
                idx = np.argwhere(prev_ids==prev_id)[0]

                # if more than one match, just select either as they have same value;
                if idx.shape[0]>0:
                    idx=idx[0]
                tracker[ctr_k+1][ctr]=tracker[ctr_k][idx]

            # start a new chain with a new ID
            else:

                next_id = find_next_id(tracker)

                next_id = max_network_id 

                tracker[ctr_k+1][ctr]=next_id

                max_network_id +=1
                print ("frame: ", k, "new val: ", next_id)

            if prev_id != None and plotting:
                #plt.plot([k-1, k], 
                #         [prev_id, current_id])
                ax1.plot([k-1, k], 
                         [tracker[ctr_k][idx], tracker[ctr_k+1][ctr]],
                        linewidth=4,
                        c=clrs[tracker[ctr_k][idx]%10])

#                 ax2.plot([k-1, k], 
#                          [tracker[ctr_k][idx], tracker[ctr_k+1][ctr]],
#                         linewidth=4,
#                         c=clrs[tracker[ctr_k][idx]%4])
#                          #c=clrs[prev_id])
    #             chains.append(
    #                         [[k-1, k], 
    #                          [prev_id, current_id]])

    #             cc2[prev_id, current_id, k-1]+=1
                #chains[]
        print ("TRACKER: ",ctr_k,  tracker[ctr_k+1])
        #print ('')

    tracker = np.array(tracker)
    if plotting:
        ax1.set_xlabel("Frame #", fontsize=50)
        ax1.tick_params(labelsize=50, length=6, width=2, colors='black',
                       grid_color='r', grid_alpha=0.5)
        #ax1.set_xticks([])
        ax1.tick_params(labelsize=50, length=6, width=2, colors='black',
                       grid_color='r', grid_alpha=0.5)
        ax1.set_ylabel("CC Network ID", fontsize=50)
        ax1.set_ylabel("CC Network ID", fontsize=50)
        ax1.set_xlim(0,frames[-1])
        ax1.set_xlim(0,frames[-1])
        #plt.savefig('/home/cat/networks1.png')
        #plt.close()
        plt.show()

    np.save(fname_tracker, 
            tracker)
else:
    tracker = np.load(fname_tracker, allow_pickle=True)

print ("DONE LOADING TRACKER")


Chain IDS:  [0 3 4 5]
PLOTING:  True
frame:  1 new val:  6
TRACKER:  0 [[], 6, 3, 4, 5]
TRACKER:  1 [[], 6, 3, 4, 5]
frame:  3 new val:  7
frame:  3 new val:  8
TRACKER:  2 [[], 7, 8, 3, 4, 5]
frame:  4 new val:  9
TRACKER:  3 [[], 9, 3, 4, 5]
TRACKER:  4 [[], 9, 3, 4, 5]
TRACKER:  5 [[], 9, 3, 4, 5]
TRACKER:  6 [[], 9, 3, 4, 5]
frame:  8 new val:  10
TRACKER:  7 [[], 10, 3, 4, 5]
frame:  9 new val:  11
TRACKER:  8 [[], 11, 3, 4, 5]
frame:  10 new val:  12
TRACKER:  9 [[], 12, 3, 4, 5]
frame:  11 new val:  13
TRACKER:  10 [[], 13, 3, 4, 5]
TRACKER:  11 [[], 13, 3, 4, 5]
frame:  13 new val:  14
TRACKER:  12 [[], 14, 3, 4, 5]
frame:  14 new val:  15
TRACKER:  13 [[], 15, 3, 4, 5]
frame:  15 new val:  16
frame:  15 new val:  17
TRACKER:  14 [[], 16, 3, 17, 4, 5]
frame:  16 new val:  18
TRACKER:  15 [[], 18, 3, 4, 5]
frame:  17 new val:  19
TRACKER:  16 [[], 19, 3, 4, 5]
TRACKER:  17 [[], 19, 3, 4, 5]
TRACKER:  18 [[], 19, 3, 4, 5]
frame:  20 new val:  20
TRACKER:  19 [[], 20, 3, 4, 5]
fra

frame:  254 new val:  94
TRACKER:  253 [[], 76, 94, 88, 5]
TRACKER:  254 [[], 88, 76, 5]
TRACKER:  255 [[], 88, 76, 5]
TRACKER:  256 [[], 88, 76, 5]
frame:  258 new val:  95
frame:  258 new val:  96
TRACKER:  257 [[], 95, 88, 96, 5]
frame:  259 new val:  97
TRACKER:  258 [[], 88, 97, 5]
frame:  260 new val:  98
frame:  260 new val:  99
TRACKER:  259 [[], 98, 88, 99, 5]
frame:  261 new val:  100
TRACKER:  260 [[], 100, 88, 5]
TRACKER:  261 [[], 100, 88, 5]
TRACKER:  262 [[], 100, 88, 5]
TRACKER:  263 [[], 100, 88, 5]
frame:  265 new val:  101
frame:  265 new val:  102
TRACKER:  264 [[], 101, 88, 102, 5]
frame:  266 new val:  103
TRACKER:  265 [[], 88, 103, 5]
TRACKER:  266 [[], 88, 103, 5]
TRACKER:  267 [[], 88, 103, 5]
TRACKER:  268 [[], 103, 88, 5]
frame:  270 new val:  104
TRACKER:  269 [[], 103, 104, 88, 5]
TRACKER:  270 [[], 88, 103, 5]
frame:  272 new val:  105
TRACKER:  271 [[], 103, 88, 105, 5]
TRACKER:  272 [[], 103, 88, 5]
TRACKER:  273 [[], 88, 103, 5]
TRACKER:  274 [[], 103,

In [43]:
###############################################################################
########## TIME_BASED CONNECTED COMPONNET 3: SAVE IMAGES FOR CLASSIFIER #######
###############################################################################

matplotlib.use('Agg')
#plt.savefig('/tmp/test.png'
            
video_name ='/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'
original_vid = cv2.VideoCapture(video_name)

root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/'

frame_id= frames[0]
original_vid.set(cv2.CAP_PROP_POS_FRAMES, frame_id)

#print ("FRames: ", frames)
#print ("CHAINS: ", tracker)

tracker_flat = flat_list = [item for sublist in tracker for item in sublist]
min_chain = 0

width = 100
padding = 30
ctr_img = 0
#ctr_k = 0

def extract_images(frame_ids,
                   surfaces_watershed,
                  ):

    video_name ='/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'
    original_vid = cv2.VideoCapture(video_name)

    # set start at frame_id 1
    frame_start = frame_ids[0]
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, frame_start)

    for ctr_k, k in enumerate(frame_ids):

        ret, frame = original_vid.read()

        # FIND UNIQUE PATCHES
        local_ids = np.unique(surfaces_watershed[k])
        img1 = surfaces_watershed[k].copy()  # original copy; do not alter;

        for p, id_ in enumerate(local_ids):

            idx0 = np.where(surfaces_watershed[k]==id_)

            # SKIP THE DEFAULT BACKGROUND PATCH
            if idx0[0].shape[0]>500:
                continue

            chain_id = tracker[frame_ids[ctr_k]][p]

            # CHECK IF CHAIN ID BELONGS TO A LARGER CHAIN
            # SAVE ALL CHAINS FOR NOW
            if False:
                idxc = np.where(tracker_flat==chain_id)[0] 
                if idxc.shape[0]<min_chain:
                    continue

            #img = img_array1[frame_id].copy()
            img = img1.copy()

            # ZERO OUT ALL NON ID_ values for PCA ROTATION COMPUTATION
            idx1 = np.where(img1==id_)
            idx2 = np.where(img1!=id_)
            img[idx1]=1
            img[idx2]=0

            # FIND CENTRES
            centrex = int(np.nanmean(idx1[0]))*8
            centrey = int(np.nanmean(idx1[1]))*8
            #print ("cenres:", centrex, centrey,
            #      centrex//8, centrey//8)

            # GRAB CENTRED PATCH
            frame_patch = frame[centrex-width-padding:centrex+width+padding, 
                                centrey-width-padding:centrey+width+padding][:,:,1]

            # EXCLUDE PATCHES THAT ARE NOT RIGHT SIZE DUE TO CLIPPING;
            # TODO: fix this so it is not required
            if frame_patch.shape[0]!= (width*2 + padding*2):
                continue
            if frame_patch.shape[1]!= (width*2 + padding*2):
                continue


            # FIND ROTATION USING PCA
            y, x = np.nonzero(img)
            x = x - np.mean(x)
            y = y - np.mean(y)
            coords = np.vstack([x, y])

            cov = np.cov(coords)
            evals, evecs = np.linalg.eig(cov)

            sort_indices = np.argsort(evals)[::-1]
            x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
            x_v2, y_v2 = evecs[:, sort_indices[1]]

            # ROTATE PATCH
            theta = np.arctan((x_v1)/(y_v1))  

            # ROTATE RAW DATA
            try:
                frame_rotated = rotate_image(frame_patch, -theta*180/3.14159)
                frame_rotated = frame_rotated[padding:-padding, padding: -padding]
            except:
                print ("Couldn't rotate or clip image")
                continue
                pass
            #print (" frame_patch: ", frame_patch.shape, 
            #       "  frame rotated: ", frame_rotated.shape)

            #print ("   ", frame_rotated.shape)
            out_fname = root_dir+str(chain_id)+'/frame_'+str(k)+"_id_"+str(p)+'.npy'

            try: 
                os.mkdir(root_dir+str(chain_id))
            except:
                pass

            fig=plt.figure(figsize=(10,10))
            plt.imshow(frame_rotated)
            plt.xticks([])
            plt.yticks([])
            plt.savefig(out_fname[:-4]+'.png')
            plt.close()

            np.save(out_fname[:-4]+'.npy', frame_rotated)
            #np.save(out_fname, frame_rotated)

            #ctr_img+=1
            
n_cores = 6            
frame_ids_split = np.array_split(frames, n_cores)
print (frame_ids_split)
if True:
    
    parmap.map(extract_images,
               frame_ids_split,
               surfaces_watershed,
               pm_pbar=True,
               pm_processes=n_cores)
else:
    extract_images(frames, 
           surfaces_watershed)

        
print (frame.shape)
#img1 = img_array1[frame_id]

#aspect = 'equal'
#n_rows = np.unique(img1).shape[0]
#for :
#    img = img_array1[frame_id].copy()
    



[array([   0,    1,    2, ..., 1664, 1665, 1666]), array([1667, 1668, 1669, ..., 3331, 3332, 3333]), array([3334, 3335, 3336, ..., 4998, 4999, 5000]), array([5001, 5002, 5003, ..., 6665, 6666, 6667]), array([6668, 6669, 6670, ..., 8331, 8332, 8333]), array([8334, 8335, 8336, ..., 9997, 9998, 9999])]


  0%|          | 0/6 [00:08<?, ?it/s]Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/parmap/parmap.py", line 105, in _func_star_single
    **func_item_args[3])
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent cal

  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/axes/_base.py", line 2590, in draw
    self._update_title_position(renderer)
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/axes/_base.py", line 2538, in _update_title_position
    if title.get_window_extent(renderer).ymin < top:
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/text.py", line 895, in get_window_extent
    tx, ty = self._get_xy_display()
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/text.py", line 228, in _get_xy_display
    return self.get_transform().transform((x, y))
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/transforms.py", line 1405, in transform
    res = self.transform_affine(self.transform_non_affine(values))
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/matplotlib/transforms.py", line 2365, in transform_affine
    return self.get_affine().transf

KeyboardInterrupt: 

KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/cat/.conda/envs/DLC-GPU/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()


In [ ]:
# Make test arrays from all the data saved;


def load_gerbil_all_data(animal_dir,
                        animal_id):
    
    from skimage.measure import block_reduce
    
    names = ['female','male','pup1','pup2']

    import glob

    
    if True:
        data = []
        fnames = glob.glob(animal_dir+'/**/*',recursive = True)
        #print ("Fnames; ", fnames)
        for fname in fnames:
            if '.npy' in fname:
                data.append(np.load(fname))
        data = np.array(data)
        print ("data: ", 
               data.shape)

    x_test = []
    y_test = []
    #data = np.load(animal_dir+"id_"+str(k)+'.npy')
        
    # scale data down:
    #image = np.arange(3*3*4).reshape(3, 3, 4)
    data_downsampled = []
    for p in range(data.shape[0]):
        temp = data[p]
       # print ("temp start: ", temp.shape)
        temp = block_reduce(data[p], 
                            block_size=(6,6), 
                            func=np.mean)
        #print ("temp finish: ", temp.shape)
        data_downsampled.append(temp)
    x_test = np.array(data_downsampled)
    print (x_test.shape)
    y_test = np.zeros(x_test.shape[0])+ animal_id
    
    #x_test = np.vstack(x_test)
    x_test = np.int32((x_test,x_test,x_test)).transpose(1,2,3,0)
    y_test = np.hstack(y_test)[:,None]
        
    print ("x test: ", x_test.shape,
           "y test: ", y_test.shape)
    return x_test, y_test





In [20]:
# 
# def save_cropped_images(frames, 
#                        surfaces_watershed,
#                        tracker):
      
#     video_name ='/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'
#     original_vid = cv2.VideoCapture(video_name)

#     root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/'

#     # LOAD NEXT VIDEO FRAME; MUST ASSIGN IN SEQUENCE SO iterator can work properly
#     frame_id = frames[0]
#     #print (frames, frame_id)
#     original_vid.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
    
#     # loop over asigned frames
#     ctr_img=0
#     for ctr_k, k in enumerate(tqdm(frames)):

#         ret, frame = original_vid.read()

#         # FIND UNIQUE PATCHES
#         local_ids = np.unique(surfaces_watershed[k])
#         img1 = surfaces_watershed[k].copy()  # original copy; do not alter;

#         for p, id_ in enumerate(local_ids):

#             idx0 = np.where(surfaces_watershed[k]==id_)

#             # SKIP THE DEFAULT BACKGROUND PATCH
#             if idx0[0].shape[0]>500:
#                 continue

#             chain_id = tracker[ctr_k][p]

#             # CHECK IF CHAIN ID BELONGS TO A LARGER CHAIN
#             idxc = np.where(tracker_flat==chain_id)[0]
#             if idxc.shape[0]<min_chain:
#                 continue

#             #img = img_array1[frame_id].copy()
#             img = img1.copy()

#             # ZERO OUT ALL NON ID_ values for PCA ROTATION COMPUTATION
#             idx1 = np.where(img1==id_)
#             idx2 = np.where(img1!=id_)
#             img[idx1]=1
#             img[idx2]=0

#             # FIND CENTRES
#             centrex = int(np.nanmean(idx1[0]))*8
#             centrey = int(np.nanmean(idx1[1]))*8
#             #print ("cenres:", centrex, centrey,
#             #      centrex//8, centrey//8)

#             # GRAB CENTRED PATCH
#             frame_patch = frame[centrex-width-padding:centrex+width+padding, 
#                                 centrey-width-padding:centrey+width+padding][:,:,1]

#             if frame_patch.shape[0]!= (width*2 + padding*2):
#                 continue
#             if frame_patch.shape[1]!= (width*2 + padding*2):
#                 continue
#             # 
#             y, x = np.nonzero(img)
#             x = x - np.mean(x)
#             y = y - np.mean(y)
#             coords = np.vstack([x, y])

#             cov = np.cov(coords)
#             evals, evecs = np.linalg.eig(cov)

#             sort_indices = np.argsort(evals)[::-1]
#             x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
#             x_v2, y_v2 = evecs[:, sort_indices[1]]

#             # ROTATE PATCH
#             theta = np.arctan((x_v1)/(y_v1))  

#             # ROTATE RAW DATA
#             try:
#                 frame_rotated = rotate_image(frame_patch, -theta*180/3.14159)
#                 frame_rotated = frame_rotated[padding:-padding, padding: -padding]
#             except:
#                 print ("Couldn't rotate or clip image")
#                 continue
#                 pass
#             #print (" frame_patch: ", frame_patch.shape, 
#             #       "  frame rotated: ", frame_rotated.shape)

#             #print ("   ", frame_rotated.shape)
#             out_fname = root_dir+str(chain_id)+'/frame_'+str(k)+"_id_"+str(p)+'.npy'

#             try: 
#                 os.mkdir(root_dir+str(chain_id))
#             except:
#                 pass


#             fig=plt.figure(figsize=(10,10))
#             plt.imshow(frame_rotated)
#             plt.xticks([])
#             plt.yticks([])
#             plt.savefig(out_fname[:-4]+'.png')
#             plt.close()

#             np.save(out_fname[:-4]+'.npy', frame_rotated)
#             #np.save(out_fname, frame_rotated)



#             ctr_img+=1
        
# matplotlib.use('Agg')
# #plt.savefig('/tmp/test.png'
            
# video_name ='/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'
# original_vid = cv2.VideoCapture(video_name)

# root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/'

# frame_id= frames[0]
# original_vid.set(cv2.CAP_PROP_POS_FRAMES, frame_id)

# #print ("FRames: ", frames)
# #print ("CHAINS: ", tracker)

# tracker_flat = flat_list = [item for sublist in tracker for item in sublist]
# min_chain = 50

# width = 100
# padding = 30
# ctr_img = 0
# #ctr_k = 0


# import parmap
# n_proc=6
# frames_split = np.array_split(frames, n_proc)
# #print (frames_split)
    
# if True:
#     parmap.map(save_cropped_images,
#                frames_split, 
#            surfaces_watershed,
#            tracker,
#            pm_pbar=True,
#            pm_processes=n_proc)
# else:
#     save_cropped_images(frames,
#             surfaces_watershed,
#            tracker)

100%|██████████| 6/6 [01:49<00:00, 18.28s/it]


IndexError: index 4 is out of bounds for axis 0 with size 4

In [27]:
######################################################################
############## MAKE CLASSIFIER TRAINING DATA #########################
######################################################################
import numpy as np

root_animals = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images_cc/segments_training/'

names = ['female','male','pup1','pup2', 'block']

import glob

# RECURSIVE LOADING OF ALL DATA
data = []
for ctr, name in enumerate(names):
    data.append([])

    fnames = glob.glob(root_animals+name+'/**/*',recursive = True)
    for fname in fnames:
        if '.npy' in fname:
            data[ctr].append(np.load(fname))

# SAVE THE DATASETS TO DISK          
for k in range(len(names)):
    print (names[k], len(data[k]))
    temp = np.array(data[k])
    print (temp.shape)
    idx_rand = np.random.choice(np.arange(temp.shape[0]), 
                                #595, 
                                int(temp.shape[0]*.9),
                                replace=False)
    print (k, idx_rand.shape)
    temp = np.array(temp[idx_rand])
    print (temp.shape)
    np.save(root_animals+"id_"+str(k)+'_train.npy', temp)
    
    # save test datasets; 
    temp = np.array(data[k])
    idx = np.arange(temp.shape[0])
    idx_test = np.delete(idx, idx_rand)
    print ("Idx test: ", idx_test.shape)
    
    temp = temp[idx_test]
    print ("saved test: ", temp.shape)
    np.save(root_animals+"id_"+str(k)+'_test.npy', temp)          

female 2922
(2922, 200, 200)
0 (2629,)
(2629, 200, 200)
Idx test:  (293,)
saved test:  (293, 200, 200)
male 1416
(1416, 200, 200)
1 (1274,)
(1274, 200, 200)
Idx test:  (142,)
saved test:  (142, 200, 200)
pup1 2113
(2113, 200, 200)
2 (1901,)
(1901, 200, 200)
Idx test:  (212,)
saved test:  (212, 200, 200)
pup2 1154
(1154, 200, 200)
3 (1038,)
(1038, 200, 200)
Idx test:  (116,)
saved test:  (116, 200, 200)
block 540
(540, 200, 200)
4 (486,)
(486, 200, 200)
Idx test:  (54,)
saved test:  (54, 200, 200)


(100, 100, 50)


In [30]:
# VISUALIZE MATCHES
clrs = ['red','blue','cyan','green','yellow','pink','magenta','white','lightgreen','lightblue']


# for k in frames:  #skip first match
#     print ("frame: ", k)
#     im = surfaces_watershed[k]
#     idx0 = np.where(im==0)
#     #print ("idx0: ", idx0)
#     im[idx0]==np.nan
    
#     # 
#     ids = np.unique(im)
#     #print (ids)
#     for ctr, id_ in enumerate(ids):
#         if id_==0:
#             continue

#         #
#         idx = np.where(im==id_)
#         centrex = np.mean(idx[0])
#         centrey = np.mean(idx[1])
#         print (centrex.shape, centrex)
        
#         #centrey = np.mean(im[idx[:,1]])

#         match_id = np.array(matches[k][ctr]).squeeze()
#         print ("matchid: ", match_id)
        
#         plt.scatter(k, centrex+centrey, 
#                     c=clrs[match_id%10])
        
# plt.show()
        
        

In [215]:
# id_ = 145
# print (np.unique(surfaces_watershed[id_-1]))
# print (np.unique(surfaces_watershed[id_]))
# print (matches[id_])


[0 4 6 7]
[0 3 5 6]
[[0], [4], [6], [7]]


In [31]:
# ##################################################################
# #################### EXTRACT PATCHES AND ALIGN THEM ##############
# ##################################################################

# #import skimage.filter


# fig=plt.figure()


# #img = misc.imread('oval.png', flatten=1)

# video_name ='/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'


# original_vid = cv2.VideoCapture(video_name)

# frame_id= 900
# original_vid.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
# ret, frame = original_vid.read()

# img1 = img_array1[frame_id]

# aspect = 'equal'
# n_rows = np.unique(img1).shape[0]
# for ctr_id, id_ in enumerate(np.unique(img1)):
    
#     img = img_array1[frame_id].copy()
    
#     #id_ = 5
#     print ("ctr_ID: ", ctr_id)

#     # PLOT FRAME
#     ax=plt.subplot(n_rows,5,1+ctr_id*5)
#     plt.imshow(frame)
#     print (frame.shape)
    
#     # PLOT FRAME 
#     ax=plt.subplot(n_rows,5,2+ctr_id*5)
#     plt.imshow(img, aspect=aspect)

    
#     # ZERO OUT ALL NON ID_ values for PCA ROTATION COMPUTATION
#     print (np.unique(img1))
#     idx1 = np.where(img1==id_)
#     idx2 = np.where(img1!=id_)
#     img[idx1]=1
#     img[idx2]=0


#     width = 125
#     padding = 30
#     print ("idx1: ", idx1[0],
#                     idx1[1])
#     centrex = int(np.nanmean(idx1[0]))*8
#     centrey = int(np.nanmean(idx1[1]))*8
#     print ("cenres:", centrex, centrey,
#           centrex//8, centrey//8)

#     #frame_patch = np.zeros((200,200),'int32')
#     frame_patch = frame[centrex-width-padding:centrex+width+padding, centrey-width-padding:centrey+width+padding][:,:,1]

#     ax=plt.subplot(n_rows,5,3+ctr_id*5)
#     plt.imshow(frame_patch,aspect=aspect)
    
    
#     #
#     y, x = np.nonzero(img)
#     x = x - np.mean(x)
#     y = y - np.mean(y)
#     coords = np.vstack([x, y])

#     cov = np.cov(coords)
#     evals, evecs = np.linalg.eig(cov)

#     sort_indices = np.argsort(evals)[::-1]
#     x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
#     x_v2, y_v2 = evecs[:, sort_indices[1]]


#     # PLOT THE PATCH WITH ANIMAL IN IT
#     ax=plt.subplot(n_rows,5,4+ctr_id*5)
#     scale = 20
#     plt.plot([x_v1*-scale*2, x_v1*scale*2],
#              [y_v1*-scale*2, y_v1*scale*2], color='red')
#     plt.plot([x_v2*-scale, x_v2*scale],
#              [y_v2*-scale, y_v2*scale], color='blue')
#     plt.plot(x, y, 'k.')
#     plt.axis('equal')
#     plt.gca().invert_yaxis()  # Match the image system with origin at top left

#     #fig=plt.figure()

#     theta = np.arctan((x_v1)/(y_v1))  
#     rotation_mat = np.matrix([[np.cos(theta), -np.sin(theta)],
#                           [np.sin(theta), np.cos(theta)]])
#     transformed_mat = rotation_mat * coords

#     # plot the transformed blob
#     x_transformed, y_transformed = transformed_mat.A
#     plt.plot(x_transformed, y_transformed, 'g.')


#     # ROTATE RAW DATA
#     ax=plt.subplot(n_rows, 5,5+ctr_id*5)
#     try: 
#         frame_rotated = rotate_image(frame_patch, -theta*180/3.14159)
#         frame_rotated = frame_rotated[padding:-padding, padding: -padding]
#         plt.imshow(frame_rotated,aspect=aspect)
#     except:
#         pass
# plt.show()


In [227]:
##############################################################
# #################### REMOVE PATCHES BELOW THRESHOLD ##############
# ##################################################################

# centres = []
# ids = []
# min_size = 0
# for k in range(102):
#     idx = np.unique(img_array[k])
#     centres.append([])
#     ids.append([])
#     #print (k, idx.shape, idx)
#     ctr=0
#     for id_ in idx:
#         idx2 = np.where(img_array[k]==id_)
#         if idx2[0].shape[0]< min_size:
#             img_array[k][idx2]=0
#             continue
#         ids[k].append(ctr)
#         if idx2[0].shape[0]>1:
#             centres[k].append(np.array([np.mean(idx2[0]), np.mean(idx2[1])]))
#         else:
#             centres[k].append(np.array([idx2[0], idx2[1]]))
#         ctr+=1    
            
#         #print (centres[k])
#     #print('')

In [32]:
# # LINK CENTRES ACROSS TIME
# centres = np.array(centres)
# print (centres.shape)
# from scipy.spatial import distance

# matches = []
# for k in range(1,102,1):
    
#     # find min distances 
#     dist = distance.cdist(centres[k],centres[k-1]) 
#     print (k, dist.min(axis=1))
#     match = dist.argmin(axis=1)
#     print ("k, ", k, "  match: ",  match)
#     matches.append(match)
    
# matches = np.array(matches)
# print (matches.shape)

In [33]:
# # FIND CENTRES FOR ALL THE CONTOURS
# #centres = []
# #min_size = 100
# imgs = img_array.copy()
# fig=plt.figure(figsize=(20,20))
# ctr_plot=0
# for k in range(20,40,1):
#     ax = plt.subplot(4,5,ctr_plot+1)
#     #idx = np.unique(img_array[k])
#     #print (k, idx.shape, idx)
#     ctr=0
#     for id_ in ids[k]:
#         idx2 = np.where(img_array[k]==id_)
#         #if idx2[0].shape[0]< min_size:
#         #    continue
#         print ("frame: ", k, " animal id: ", id_, " ctr: ", ctr)
#         print ("ids ", ids[k])
#         print ("matches ", matches[k])
#         print ("   matches ", matches[k-1][ctr])
#         #print (matches[k])
#         imgs[k][idx2]=matches[k-1][ctr]
#         ctr+=1
        
#     plt.ylabel(str(k))
#     idx =np.where(imgs[k]<0)
#     imgs[k][idx]=0
    
#     vmin = np.min(imgs[k]); vmax=np.max(imgs[k])
#     img2 = surfaces[k]
#     img2 = (img2-np.min(img2))/(np.max(img2)-np.min(img2))
#     img2 = img2*(vmax-vmin)+vmin
#     img_final = np.vstack((imgs[k],img2))
#     img_final[img_final.shape[0]//2]=np.nan
#     plt.imshow(img_final)
#     plt.xticks([])
#     plt.yticks([])
#     ctr_plot+=1
    
# #plt.show()
# #"/home/cat/watershed.png",dpi=1200)
# fig.savefig("/home/cat/watershed.svg", format='svg', dpi=300)
# plt.close()

In [176]:
# frame = 24
# ax =plt.subplot(1,2,1)
# plt.imshow(img_array[frame])
# ax =plt.subplot(1,2,2)
# plt.imshow(surfaces[frame])

# plt.show()